# Qiskit VQE - SU(2) Matrix Model [Lambda = 4]

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal, EfficientSU2

import sys
sys.path.append('../../utility')
from vqe_run import *
from qc_ansatze import *

# Convert the Hamiltonian to strings of Pauli ops

In [2]:
# Load the H file already created using qiskit 0.28 from MBP 2015
file = open("../../utility/pauliH_L4_g2.0.txt", "r")
content = file.read()
#print(content)

In [3]:
data = content.split(',\n')
data[0] = data[0].split('\n')[1]
data[-1] = data[-1].split('\n')[0]

data_coeff = []
data_str = []
for i in range(len(data)):
    data_coeff.append(float(data[i].split('*')[0]))
    data_str.append(data[i].split('*')[1].split(' ')[1])
    
Hpauli = list(zip(data_str,data_coeff))
#Hpauli

In [4]:
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms import NumPyEigensolver
H4q = SparsePauliOp.from_list(Hpauli)

solver = NumPyEigensolver(k=4)
exact_solution = solver.compute_eigenvalues(H4q)
print("Exact Result of qubit hamiltonian:", np.real(exact_solution.eigenvalues))
E_exact = np.round(np.real(exact_solution.eigenvalues)[0],5)
E_exact

Exact Result of qubit hamiltonian: [3.89547837 5.2753731  5.2753731  5.2753731 ]


3.89548

# Circuit ansatze

In [5]:
#CIRCUIT ANSATZ
from qiskit.circuit.library import TwoLocal, EfficientSU2

In [6]:
reps=1
ansatz_0a = TL_ansatz(H4q.num_qubits, 'ry', 'crx', "circular", reps)
ansatz_0b = TL_ansatz(H4q.num_qubits, 'rz', 'crx', 'circular', reps)
ansatz_0c = TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', 'circular', reps)
ansatz_0d = TL_ansatz(H4q.num_qubits,['ry','y'], 'crx', 'circular', reps)

ansatz_1a =  TL_ansatz(H4q.num_qubits, 'ry', 'crx', "full", reps)
ansatz_1b =  TL_ansatz(H4q.num_qubits, 'rz', 'crx', "full", reps)
ansatz_1c =  TL_ansatz(H4q.num_qubits, ['ry','rz'], 'crx', "full", reps)
ansatz_1d =  TL_ansatz(H4q.num_qubits, ['ry','y'], 'crx', "full", reps)

Circuit ansatz with 36 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 60 parameters
Circuit ansatz with 36 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 90 parameters
Circuit ansatz with 114 parameters
Circuit ansatz with 90 parameters


In [7]:
reps=1
ansatz_0a_su2 = ef_ansatz(H4q.num_qubits, 'ry',  "circular", reps)
ansatz_0b_su2 = ef_ansatz(H4q.num_qubits, 'rz',  'circular', reps)
ansatz_0c_su2 = ef_ansatz(H4q.num_qubits, ['ry','rz'], 'circular', reps)
ansatz_0d_su2 = ef_ansatz(H4q.num_qubits,['ry','y'], 'circular', reps)

ansatz_1a_su2 =  ef_ansatz(H4q.num_qubits, 'ry',  "full", reps)
ansatz_1b_su2 =  ef_ansatz(H4q.num_qubits, 'rz',  "full", reps)
ansatz_1c_su2 =  ef_ansatz(H4q.num_qubits, ['ry','rz'], "full", reps)
ansatz_1d_su2 =  ef_ansatz(H4q.num_qubits, ['ry','y'],  "full", reps)

Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 24 parameters
Circuit ansatz with 48 parameters
Circuit ansatz with 24 parameters


# Qiskit QVE: Noiseless simulator

https://qiskit.github.io/qiskit-aer/stubs/qiskit_aer.primitives.Estimator.html

https://qiskit-community.github.io/qiskit-algorithms/tutorials/03_vqe_simulation_with_noise.html

In [8]:
#VQE ALGORITHMS
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals 
from qiskit_aer.primitives import Estimator as AerEstimator

import warnings
warnings.filterwarnings("ignore")
seed = 170
iterations = 450
algorithm_globals.random_seed = seed

#estimator
noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)
#storing values
counts = []
values = []
def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
def run_qve_w_specified_optimizer(optimizer, ansatz):
    opt = optimizer(maxiter = iterations)
    vqe = VQE(noiseless_estimator, ansatz, optimizer=opt, callback=store_intermediate_result)
    result = vqe.compute_minimum_eigenvalue(operator=H4q).eigenvalue.real
    print(f"VQE result: {result:.5f}")
    return result

# COBYLA: EffSU2

In [9]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a_su2)
counts_0a_c = counts
values_0a_c = values

VQE result: 10.89956


In [10]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b_su2)
counts_0b_c = counts
values_0b_c = values

VQE result: 4.79100


In [11]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c_su2)
counts_0c_c = counts
values_0c_c = values

VQE result: 10.98084


In [12]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d_su2)
counts_0d_c = counts
values_0d_c = values

VQE result: 9.68253


In [13]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a_su2)
counts_1a_c = counts
values_1a_c = values

VQE result: 8.01434


In [14]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b_su2)
counts_1b_c = counts
values_1b_c = values

VQE result: 4.79100


In [15]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c_su2)
counts_1c_c = counts
values_1c_c = values

VQE result: 7.25607


In [16]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d_su2)
counts_1d_c = counts
values_1d_c = values

VQE result: 7.79339


In [17]:
df1 = pd.DataFrame({'values_0a_su2': values_0a_c})
df2 = pd.DataFrame({'values_0b_su2': values_0b_c})
df3 = pd.DataFrame({'values_0c_su2': values_0c_c})
df4 = pd.DataFrame({'values_0d_su2': values_0d_c})
df5 = pd.DataFrame({'values_1a_su2': values_1a_c})
df6 = pd.DataFrame({'values_1b_su2': values_1b_c})
df7 = pd.DataFrame({'values_1c_su2': values_1c_c})
df8 = pd.DataFrame({'values_1d_su2': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=2.0_effsu2_cobyla.csv')
qve_l2_tl_cobyla

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,25.215032,4.791,23.383645,13.483215,23.089486,4.791,24.512386,21.012662
1,29.751984,4.791,22.964273,12.344581,26.429760,4.791,25.218927,20.070572
2,24.857254,4.791,23.437586,13.373705,24.578216,4.791,27.433128,18.814374
3,26.008487,4.791,24.236187,12.975155,26.493191,4.791,22.558490,19.302091
4,21.647651,4.791,24.021724,14.215457,23.287033,4.791,23.253429,17.282560
...,...,...,...,...,...,...,...,...
445,NaN,NaN,11.054407,NaN,NaN,NaN,7.288365,NaN
446,NaN,NaN,11.013675,NaN,NaN,NaN,7.288954,NaN
447,NaN,NaN,11.076694,NaN,NaN,NaN,7.406267,NaN
448,NaN,NaN,10.991668,NaN,NaN,NaN,7.272051,NaN


# COBYLA: TwoLocal

In [18]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0a)
counts_0a_c = counts
values_0a_c = values

VQE result: 6.39657


In [19]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0b)
counts_0b_c = counts
values_0b_c = values

VQE result: 4.79100


In [20]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0c)
counts_0c_c = counts
values_0c_c = values

VQE result: 7.28931


In [21]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_0d)
counts_0d_c = counts
values_0d_c = values

VQE result: 4.24579


In [22]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1a)
counts_1a_c = counts
values_1a_c = values

VQE result: 4.06366


In [23]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1b)
counts_1b_c = counts
values_1b_c = values

VQE result: 4.79100


In [24]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1c)
counts_1c_c = counts
values_1c_c = values

VQE result: 7.37021


In [25]:
counts = []
values = []
result = run_qve_w_specified_optimizer(COBYLA, ansatz_1d)
counts_1d_c = counts
values_1d_c = values

VQE result: 3.96265


In [26]:
df1 = pd.DataFrame({'values_0a': values_0a_c})
df2 = pd.DataFrame({'values_0b': values_0b_c})
df3 = pd.DataFrame({'values_0c': values_0c_c})
df4 = pd.DataFrame({'values_0d': values_0d_c})
df5 = pd.DataFrame({'values_1a': values_1a_c})
df6 = pd.DataFrame({'values_1b': values_1b_c})
df7 = pd.DataFrame({'values_1c': values_1c_c})
df8 = pd.DataFrame({'values_1d': values_1d_c})
qve_l2_tl_cobyla = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8], axis = 1)
qve_l2_tl_cobyla.to_csv('qve_l4_l=2.0_twolocal_cobyla.csv')
qve_l2_tl_cobyla

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,14.651227,4.791,30.240499,34.048799,32.315205,4.791,31.728634,32.918796
1,15.540178,4.791,29.241185,33.591876,32.329584,4.791,29.226991,32.898954
2,16.365019,4.791,31.019843,31.867733,30.809798,4.791,32.900218,30.189507
3,14.692054,4.791,29.474552,31.853879,31.241966,4.791,29.301541,28.711147
4,17.915452,4.791,29.768351,30.578290,32.471985,4.791,25.766603,30.844485
...,...,...,...,...,...,...,...,...
445,NaN,NaN,7.668333,4.280775,4.497009,4.791,7.855225,4.246454
446,NaN,NaN,7.636879,4.289242,4.319980,4.791,7.625272,4.045527
447,NaN,NaN,7.427765,4.263075,4.367754,4.791,7.913050,4.159242
448,NaN,NaN,7.536880,4.265080,4.063658,4.791,7.541435,4.010364


# SPSA: EffSU2

In [27]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a_su2)
counts_0a = counts
values_0a = values

VQE result: 6.92786


In [28]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b_su2)
counts_0b = counts
values_0b = values

VQE result: 4.79100


In [29]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c_su2)
counts_0c = counts
values_0c = values

VQE result: 4.32874


In [30]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d_su2)
counts_0d = counts
values_0d = values

VQE result: 4.94393


In [31]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a_su2)
counts_1a = counts
values_1a = values

VQE result: 4.20670


In [32]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b_su2)
counts_1b = counts
values_1b = values

VQE result: 4.79100


In [33]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c_su2)
counts_1c = counts
values_1c = values

VQE result: 5.87643


In [34]:
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d_su2)
counts_1d = counts
values_1d = values

VQE result: 5.54155


In [35]:
# save the data to a dataframe 
list_l2 = {'values_0a_su2': values_0a, 'values_0b_su2': values_0b, 
        'values_0c_su2': values_0c, 'values_0d_su2': values_0d,
        'values_1a_su2': values_1a, 'values_1b_su2': values_1b, 
        'values_1c_su2': values_1c,'values_1d_su2': values_1d}
qve_l2_efsu2_spsa= pd.DataFrame(list_l2)
qve_l2_efsu2_spsa.to_csv('qve_l4_l=2.0_effsu2_spsa.csv')
qve_l2_efsu2_spsa

,values_0a_su2,values_0b_su2,values_0c_su2,values_0d_su2,values_1a_su2,values_1b_su2,values_1c_su2,values_1d_su2
0,22.972602,4.067432,25.688852,21.768041,19.216249,4.067432,21.276864,26.975797
1,24.964117,3.788331,25.693471,18.502454,18.043652,3.788331,23.363185,32.613269
2,25.019979,4.829427,26.544672,21.336614,18.709553,4.829427,23.084185,28.894893
3,25.510435,5.118578,23.995168,18.270035,19.481752,5.118578,22.860638,31.505875
4,25.578407,4.250873,24.581120,18.882009,20.684999,4.250873,23.119165,33.497943
...,...,...,...,...,...,...,...,...
946,6.733101,4.793952,5.404564,5.126428,4.741554,4.793952,6.096327,5.629713
947,7.636435,4.735870,5.093858,4.361708,4.603607,4.735870,6.035681,5.741495
948,7.056390,4.793952,5.289359,4.567585,4.448496,4.793952,6.382048,5.630559
949,7.440752,4.735870,5.070021,4.673984,4.772700,4.735870,5.485871,5.356617


# SPSA: TwoLocal

In [36]:
#3.18339
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0a)
counts_0a = counts
values_0a = values

VQE result: 5.45181


In [37]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0b)
counts_0b = counts
values_0b = values

VQE result: 4.79100


In [38]:
#3.62318
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0c)
counts_0c = counts
values_0c = values

VQE result: 4.61735


In [39]:
#3.13679
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_0d)
counts_0d = counts
values_0d = values

VQE result: 3.94466


In [40]:
#3.16617
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1a)
counts_1a = counts
values_1a = values

VQE result: 6.29251


In [41]:
#3.17910
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1b)
counts_1b = counts
values_1b = values

VQE result: 4.79100


In [42]:
#4.10208
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1c)
counts_1c = counts
values_1c = values

VQE result: 5.42990


In [43]:
#4.26060
counts = []
values = []
result_spsa = run_qve_w_specified_optimizer(SPSA, ansatz_1d)
counts_1d = counts
values_1d = values

VQE result: 5.61301


In [44]:
# save the data to a dataframe 
#don't reevaluate on a new run with just the new ansatze added
import pandas as pd
list_l2 = {'values_0a': values_0a, 'values_0b': values_0b, 
        'values_0c': values_0c, 'values_0d': values_0d,
        'values_1a': values_1a, 'values_1b': values_1b, 
        'values_1c': values_1c,'values_1d': values_1d}
qve_l2_tl_spsa= pd.DataFrame(list_l2)
qve_l2_tl_spsa.to_csv('qve_l4_l=2.0_twolocal_spsa.csv')
qve_l2_tl_spsa

,values_0a,values_0b,values_0c,values_0d,values_1a,values_1b,values_1c,values_1d
0,18.054421,4.067432,33.927065,16.170605,24.281109,4.067432,21.216333,31.791125
1,27.279837,3.788331,30.977329,19.065909,23.203049,3.788331,23.806564,30.672371
2,21.148671,4.829427,31.382094,13.945755,21.518349,4.829427,20.989806,31.350249
3,24.219643,5.118578,32.985096,20.623285,26.171830,5.118578,23.345216,33.498207
4,28.175374,4.250873,31.045547,18.509588,22.193999,4.250873,24.791514,32.362149
...,...,...,...,...,...,...,...,...
946,5.785361,4.793952,4.512135,4.433265,6.243568,4.793952,5.711119,5.728416
947,5.654489,4.735870,5.073201,4.167224,6.095527,4.735870,5.643926,5.655118
948,6.079271,4.793952,4.064530,4.559193,6.521574,4.793952,5.549784,5.584763
949,5.233241,4.735870,4.680679,4.068305,6.247818,4.735870,5.902090,5.065779
